# Screening Ecoinvent market consumption mixes: grouping by ISIC categories

In [12]:
import brightway2 as bw
import bw2calc as bc
import bw2data as bd
import numpy as np
import pandas as pd
import presamples as ps
import scipy
import scipy.stats as stats
from code.functions import make_dirichlet_distribution, get_elec_input, create_presamples, query_for_activities
ps.__version__

(0, 2, 6)

In [13]:
from presamples import array

def sequential_sample(self, index):
    """Draw a new sample from the pre-sampled arrays sequentially not randomly. The original code draws samples randomly. """
    index_ordered = self.count % self.data[0].shape[1]
    result = np.hstack([arr[:, index_ordered] for arr in self.data])
    self.count += 1   
    return result


array.RegularPresamplesArrays.sample = sequential_sample

In [14]:
print(array.RegularPresamplesArrays.sample.__doc__)

Draw a new sample from the pre-sampled arrays sequentially not randomly. The original code draws samples randomly. 


In [15]:
bw.projects.set_current("UK-wood-clca")

cutoff391 = bw.Database('cutoff-3.9.1')
fus = pd.read_excel(r"run.xlsx", sheet_name='to_run')
numbers = fus['indices'].tolist()
output_dir  = fus['names'].tolist()
amount  = fus['amount'].tolist()
reference_product  = fus['reference_product'].tolist()
reference_product_index = fus['reference_product_index'].tolist()
tier = fus['tier'].tolist()

fu = [cutoff391.get({num}) for num in numbers]


methods = [

     ('ReCiPe 2016 v1.03, midpoint (H)', 'climate change', 'global warming potential (GWP1000)'), 

]

In [16]:
for i in range(len(fu)):
    print(fu[i])

'market for sawnwood, board, softwood, raw, dried (u=20%)' (cubic meter, CH, None)


In [17]:
i = 0

In [18]:
output_dir

['_market_for_sawnwood__board__softwood__raw__dried__u_20_____cubic_meter__CH__None_']

In [19]:
from code.morris_category_analysis import perform_morris_category_analysis

In [20]:
%%time
levels = [12]
trajectories = [50]
Global_category_scores = []
for i in range(len(output_dir)): 
    for level in levels:
        if output_dir != "_esters_of_versatic_acid_production___kilogram__RoW__None_'":
            sa_category = perform_morris_category_analysis(fu[i], 
                                         amount[i],
                                        reference_product[i],
                                        reference_product_index[i],
                                        tier[i],
                                         output_dir[i], 
                                         methods, 
                                         cutoff391, 
                                         trajectories = 50, 
                                         num_levels =level ,
                                         mean = 117.8113726,
                                         market_dir=output_dir[i])
            sa_category['fu'] = output_dir[i]
            sa_category['levels'] =level
            Global_category_scores.append(sa_category)
    

322
188
[{'tech_indices': [2658, 8384], 'activity': 'market for electricity, high voltage' (kilowatt hour, JP, None), 'exchange': 'electricity production, natural gas, conventional power plant' (kilowatt hour, JP, None), 'database': 'cutoff-3.9.1', 'value': 0.22923296689987183, 'category': '3510:Electric power generation, transmission and distribution', 'group_market': 0}, {'tech_indices': [3020, 8384], 'activity': 'market for electricity, high voltage' (kilowatt hour, JP, None), 'exchange': 'electricity production, hydro, reservoir, alpine region' (kilowatt hour, JP, None), 'database': 'cutoff-3.9.1', 'value': 0.018638895824551582, 'category': '3510:Electric power generation, transmission and distribution', 'group_market': 0}, {'tech_indices': [3926, 8384], 'activity': 'market for electricity, high voltage' (kilowatt hour, JP, None), 'exchange': 'electricity production, hydro, pumped storage' (kilowatt hour, JP, None), 'database': 'cutoff-3.9.1', 'value': 0.009313433431088924, 'catego

1882
CPU times: total: 7min 11s
Wall time: 6min 51s


In [21]:
Global_category_scores_to_save = pd.concat(Global_category_scores, ignore_index=True)

In [22]:
Global_category_scores_to_save = Global_category_scores_to_save[Global_category_scores_to_save['mu_star_relative'] > 4]

In [23]:
Global_category_scores_to_save.to_csv(r"results/Global__market_categories_4_products_to_detailed_analysis.csv", index=False)